<a id='sec0'></a>

# Writing n-class word extraction functions

<a href='#sec1'><b>1. Extraction by frequency differences between classes</b></a>
   - Thresholds for minimum fraction of documents appearing or per-doc frequency
   - Threshold with absolute difference (by subtraction) or relative/fold difference (by division)

<a href='#sec2'><b>2. Extraction by exclusive appearances in certain classes</b></a>
   - Threshold for minimum fraction of documents appearing or per-doc frequency
   
<a href='#sec3'><b>3. Classification by Random Forest Classifier</b></a>
   1. <a href='#sec3_1'>RFC with exclusive words</a>
   2. <a href='#sec3_2'>RFC with words selected by relative difference - fraction of docs appearing</a>

In [1]:
import csv
import json
import random
import re
import numpy as np
import pandas as pd
import scipy.stats as scs
import matplotlib.pyplot as plt
import xgboost as xgb
import importlib as imp
import feature_engineering.frequency_selection as fefs

from xgboost import plot_importance
from pprint import pprint
from matplotlib  import cm
from mpl_toolkits.mplot3d import Axes3D
from collections import Counter
#from gensim import corpora, matutils, models, similarities
#from nltk import PorterStemmer
#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
#from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

%matplotlib inline

Using TensorFlow backend.
/usr/lib64/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
Slow version of gensim.models.doc2vec is being used


<b>Import and pre-process data</b>

In [2]:
class_train = pd.read_csv('./data/training_variants')
text_train = pd.read_csv("./data/training_text", sep="\|\|", engine='python',
                         header=None, skiprows=1, names=["ID","Text"])
train = class_train.merge(text_train, on='ID')

# create class label container
class_labels = []
for i in range(9):
    class_labels.append('class' + str(i+1))

<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: DeprecationWarning: invalid escape sequence \|
<input>:2: Dep

In [3]:
with open('./data/classified_docs.json') as f1:
    classified_docs = json.load(f1)

with open('./data/classified_tokenized_docs.json') as f2:
    classified_tokenized_docs = json.load(f2)

with open('./data/classified_texts.json') as f3:
    classified_texts = json.load(f3)

with open('./data/classified_tokenized_texts.json') as f4:
    classified_tokenized_texts = json.load(f4)

with open('./data/average_per_document_appearances.json') as f5:
    ave_perdoc_apps = json.load(f5)
    
with open('./data/fraction_of_documents_with_appearance.json') as f6:
    app_freqs = json.load(f6)

with open("./data/unclassified_tokenized_docs.csv", "r") as f:
    reader = csv.reader(f)
    unclassified_tokenized_docs = list(reader)

perdoc_apps = pd.DataFrame(ave_perdoc_apps).fillna(value=0)

In [4]:
%%time
fracdocs = pd.DataFrame(app_freqs).fillna(value=0)
n = 3000

top_words = []
for i in range(9):
    tops = fracdocs[class_labels[i]].sort_values(ascending=False).head(n)
    top_words.append(list(tops.index))

overlap1 = set(top_words[0])
for lis in top_words[1:]:
    overlap1.intersection_update(lis)
print('# intersecting words among top%d appearing words in each class: ', len(overlap1))
    
remove_list = []
for i in range(9):
    remove_words = [word for word in overlap1 \
                    if word in fracdocs[class_labels[i]] \
                    if fracdocs[class_labels[i]][word] > 0.5]
    remove_list.append(list(remove_words))

overlap2 = set(remove_list[0])
for lis in remove_list[1:]:
    overlap2.intersection_update(lis)
print('# intersecting words with >50% appearance: ', len(overlap2))

fracdocs_update1 = fracdocs.copy()
fracdocs_update1 = fracdocs_update1.drop(overlap2)
print('Table shape before removal: ', fracdocs.shape)
print('Table shape after removal:  ', fracdocs_update1.shape)

# intersecting words among top%d appearing words in each class:  1391
# intersecting words with >50% appearance:  287
Table shape before removal:  (125448, 9)
Table shape after removal:   (125161, 9)
CPU times: user 359 ms, sys: 9.86 ms, total: 369 ms
Wall time: 368 ms


<a id='sec1'></a>
<b>Extraction by fold difference  in frequency between classes</b>
(<a href='#sec0'>Back to top</a>)

In [5]:
%%time
n_class_words = fefs.getNClassWords(fracdocs_update1, doc_type='fraction_of_docs', mode='relative',
                                min_frequency=0.35, min_difference=1.4, print_result=True)
print('\n')
n_class_words = fefs.getNClassWords(fracdocs_update1, doc_type='fraction_of_docs', mode='absolute',
                                min_frequency=0.35, min_difference=0.2, print_result=True)
print('\n')
n_class_words = fefs.getNClassWords(perdoc_apps, doc_type='per_doc_frequency', mode='relative',
                                min_frequency=2, min_difference=5, print_result=True)
print('\n')
n_class_words = fefs.getNClassWords(perdoc_apps, doc_type='per_doc_frequency', mode='absolute',
                                min_frequency=2, min_difference=3, print_result=True)

====== n-class words extractions by relative differecne ======
Input Type: fraction_of_docs
Minimum Difference = 1.40
Minimum Frequency = 0.35
# of words in one_class_words: 312
# of words in two_class_words: 117
# of words in three_class_words: 11
# of words in four_class_words: 14
# of words in five_class_words: 11
# of words in six_class_words: 6
# of words in seven_class_words: 25
# of words in eight_class_words: 69
# of words in other_words: 124581
Total # of words: 125146


====== n-class words extractions by absolute differecne ======
Input Type: fraction_of_docs
Minimum Difference = 0.20
Minimum Frequency = 0.35
# of words in one_class_words: 186
# of words in two_class_words: 56
# of words in three_class_words: 2
# of words in four_class_words: 2
# of words in five_class_words: 1
# of words in six_class_words: 1
# of words in seven_class_words: 2
# of words in eight_class_words: 16
# of words in other_words: 124885
Total # of words: 125151


====== n-class words extractions by

<a id='sec2'></a>
<b>Extraction by exclusive appearance in certain classes</b>
(<a href='#sec0'>Back to top</a>)

In [6]:
%%time
min_freq = 0.15
n_class_words_excl = fefs.getNClassExclusiveWords(fracdocs_update1, min_frequency=min_freq)

===== n-class words extractions by exclusive appearances =====
Minimum Frequency = 0.150000
# of words in one_class_words: 264
# of words in two_class_words: 4
# of words in three_class_words: 0
# of words in four_class_words: 0
# of words in five_class_words: 0
# of words in six_class_words: 0
# of words in seven_class_words: 0
# of words in eight_class_words: 3
# of words in other_words: 124890
Total # of words: 125161
CPU times: user 9.08 s, sys: 41 ms, total: 9.12 s
Wall time: 9.03 s


<a id='sec3'></a>
# 3. Classification with Random Forest Classifier
(<a href='#sec0'>Back to top</a>)

In [7]:
select_words = fefs.selectNClassWords(n_class_words_excl, 2)
print(len(select_words))

268


In [8]:
vec_result = fefs.myVectorizer(unclassified_tokenized_docs, select_words, type='count')
vec_result['matrix'].shape

(3321, 268)

In [9]:
%%time
y = np.array(class_train.Class).astype(int).ravel()

rfc = RandomForestClassifier(n_estimators=100, max_depth=50,
                             n_jobs=7, random_state=33)

fefs.RFC_NClassWords(unclassified_tokenized_docs, fracdocs_update1, y, 
                    n_class=3, doc_type='fraction_of_docs', extract_mode='relative', 
                    min_difference=1.4, min_frequency=0.35, vector_type='tfidf',
                    test_size=0.15, random_state=None,
                    rfc=rfc)

Extracting words...
440 words extracted...
Vectorizating texts...
Training the classifier...
Making predictions...
===== Prediction Result =====
 - Accuracyl: 0.623
 - Log Loss: 1.925
CPU times: user 14.2 s, sys: 91.3 ms, total: 14.3 s
Wall time: 13.4 s


In [11]:
%%time
y = np.array(class_train.Class).astype(int).ravel()

rfc = RandomForestClassifier(n_estimators=100, max_depth=50,
                             n_jobs=7, random_state=33)

fefs.RFC_NClassExclusiveWords(unclassified_tokenized_docs, fracdocs_update1, y, 
                    n_class=3, min_frequency=0.15, vector_type='tfidf',
                    test_size=0.15, random_state=None,
                    rfc=rfc)

Extracting words...
268 words extracted...
Vectorizating texts...
Training the classifier...
Making predictions...
===== Prediction Result =====
 - Accuracyl: 0.309
 - Log Loss: 1.785
CPU times: user 13.3 s, sys: 78.4 ms, total: 13.4 s
Wall time: 13.2 s


## PCA & LDA

with Tfidf vectorizer

In [ ]:
%%time
ncw_labels = ['one_class_words', 'two_class_words', 'three_class_words', 
                  'four_class_words', 'five_class_words', 'six_class_words', 
                  'seven_class_words', 'eight_class_words','other_words']

n = 2
freq = 0.35
diff = 1.4
min_freq = 0.15

nclass_words = get_nClassWords(fracdocs_update1, doc_type='fraction_of_docs', mode='relative',
                               min_frequency=freq, min_difference=diff, print_result=True)
nclass_words_excl = get_nClassExclusiveWords(fracdocs_update1, min_frequency=min_freq)

select_words = []
for i in range(n):
    select_words += nclass_words[ncw_labels[i]]
print('%d words extracted by fold difference' % len(select_words))

excl_words = nclass_words_excl[ncw_labels[0]] + nclass_words_excl[ncw_labels[1]]
print('%d exclusive words extracted' % len(excl_words))
select_words = select_words + excl_words

select_words = list(set(select_words))
print('%d unique words extracted in total' % len(select_words))

# pre-process the raw texts
print('Pre-processing texts...')
unclf_docs2 = []
for j, doc in enumerate(unclassified_tokenized_docs):
    doc = [word for word in doc if word in select_words]
    text = ' '.join(doc)
    unclf_docs2.append(text)

# Vectroize by Frequency Counts
print('Vectorizating texts...')
vectorizer = TfidfVectorizer()
#vectorizer = CountVectorizer()

X = vectorizer.fit_transform(unclf_docs2).toarray()
X = X.astype(float)
y = np.array(class_train.Class).astype(int).ravel()

print(X.shape)
print(y.shape)

In [ ]:
# Create a new table with ID as index
pca_table1 = pd.DataFrame(index=class_train.ID)
pca = PCA(n_components=3)
pca.fit(X)
pca_table1['x'] = pca.transform(X).T[0]
pca_table1['y'] = pca.transform(X).T[1]
pca_table1['z'] = pca.transform(X).T[2]

fig = plt.figure(1, figsize=(8, 8))
plt.clf()
ax = Axes3D(fig, elev=18, azim=55)#, rect=[0, 0, .95, 1])
ax.scatter(pca_table1.x, pca_table1.y, pca_table1.z, c=class_train['Class'], marker = 'o', s=30, cmap = cm.jet)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(16,5))
ax1.scatter(pca_table1.x, pca_table1.y, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA1-PCA2')
ax2.scatter(pca_table1.y, pca_table1.z, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA2-PCA3')
ax3.scatter(pca_table1.z, pca_table1.x, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA3-PCA1')

In [ ]:
# Create a new table with ID as index
lda_table1 = pd.DataFrame(index=class_train.ID)
lda = LinearDiscriminantAnalysis(n_components=3)
lda.fit(X, y)
lda_table1['x'] = lda.transform(X).T[0]
lda_table1['y'] = lda.transform(X).T[1]
lda_table1['z'] = lda.transform(X).T[2]

fig = plt.figure(1, figsize=(8, 8))
plt.clf()
ax = Axes3D(fig, elev=18, azim=55)#, rect=[0, 0, .95, 1])
ax.scatter(lda_table1.x, lda_table1.y, lda_table1.z, c=class_train['Class'], marker = 'o', s=30, cmap = cm.jet)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(16,5))
ax1.scatter(lda_table1.x, lda_table1.y, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA1-PCA2')
ax2.scatter(lda_table1.y, lda_table1.z, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA2-PCA3')
ax3.scatter(lda_table1.z, lda_table1.x, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA3-PCA1')

In [ ]:
%%time
pca = PCA()
rfc = RandomForestClassifier(n_estimators=100, max_depth=50, n_jobs=7, random_state=33)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
pca.fit(X_train, y_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print(X_train.shape, X_train_pca.shape)
print(X_test.shape, X_test_pca.shape)

rfc.fit(X_train_pca, y_train)

accuracy = accuracy_score(y_test, rfc.predict(X_test_pca))
lloss = log_loss(y_test, rfc.predict_proba(X_test_pca), labels=list(range(1, 10)))

print('Accuracy %.3f' % accuracy)
print('Log Loss %.3f' % lloss)

In [ ]:
%%time
lda = LinearDiscriminantAnalysis()
rfc = RandomForestClassifier(n_estimators=100, max_depth=50, n_jobs=7, random_state=33)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
lda.fit(X_train, y_train)
X_train_lda = lda.transform(X_train)
X_test_lda = lda.transform(X_test)
print(X_train.shape, X_train_lda.shape)
print(X_test.shape, X_test_lda.shape)

rfc.fit(X_train_lda, y_train)

accuracy = accuracy_score(y_test, rfc.predict(X_test_lda))
lloss = log_loss(y_test, rfc.predict_proba(X_test_lda), labels=list(range(1, 10)))

print('Accuracy %.3f' % accuracy)
print('Log Loss %.3f' % lloss)

with Count vectorizer, also with 2-class exclusive words

In [ ]:
%%time
ncw_labels = ['one_class_words', 'two_class_words', 'three_class_words', 
                  'four_class_words', 'five_class_words', 'six_class_words', 
                  'seven_class_words', 'eight_class_words','other_words']

n = 2
freq = 0.35
diff = 1.4
min_freq = 0.15

nclass_words = get_nClassWords(fracdocs_update1, doc_type='fraction_of_docs', mode='relative',
                               min_frequency=freq, min_difference=diff, print_result=True)
nclass_words_excl = get_nClassExclusiveWords(fracdocs_update1, min_frequency=min_freq)

select_words = []
for i in range(n):
    select_words += nclass_words[ncw_labels[i]]
print('%d words extracted by fold difference' % len(select_words))

excl_words = nclass_words_excl[ncw_labels[0]] + nclass_words_excl[ncw_labels[1]]
print('%d exclusive words extracted' % len(excl_words))
select_words = select_words + excl_words

select_words = list(set(select_words))
print('%d unique words extracted in total' % len(select_words))

# pre-process the raw texts
print('Pre-processing texts...')
unclf_docs2 = []
for j, doc in enumerate(unclassified_tokenized_docs):
    doc = [word for word in doc if word in select_words]
    text = ' '.join(doc)
    unclf_docs2.append(text)

# Vectroize by Frequency Counts
print('Vectorizating texts...')
#vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(unclf_docs2).toarray()
X = X.astype(float)
y = np.array(class_train.Class).astype(int).ravel()

print(X.shape)
print(y.shape)

In [ ]:
# Create a new table with ID as index
pca_table1 = pd.DataFrame(index=class_train.ID)
pca = PCA(n_components=3)
pca.fit(X)
pca_table1['x'] = pca.transform(X).T[0]
pca_table1['y'] = pca.transform(X).T[1]
pca_table1['z'] = pca.transform(X).T[2]

fig = plt.figure(1, figsize=(8, 8))
plt.clf()
ax = Axes3D(fig, elev=18, azim=55)#, rect=[0, 0, .95, 1])
ax.scatter(pca_table1.x, pca_table1.y, pca_table1.z, c=class_train['Class'], marker = 'o', s=30, cmap = cm.jet)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(16,5))
ax1.scatter(pca_table1.x, pca_table1.y, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA1-PCA2')
ax2.scatter(pca_table1.y, pca_table1.z, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA2-PCA3')
ax3.scatter(pca_table1.z, pca_table1.x, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA3-PCA1')

In [ ]:
# Create a new table with ID as index
lda_table1 = pd.DataFrame(index=class_train.ID)
lda = LinearDiscriminantAnalysis(n_components=3, solver='svd')
lda.fit(X, y)
lda_table1['x'] = lda.transform(X).T[0]
lda_table1['y'] = lda.transform(X).T[1]
lda_table1['z'] = lda.transform(X).T[2]

fig = plt.figure(1, figsize=(8, 8))
plt.clf()
ax = Axes3D(fig, elev=18, azim=55)#, rect=[0, 0, .95, 1])
ax.scatter(lda_table1.x, lda_table1.y, lda_table1.z, c=class_train['Class'], marker = 'o', s=30, cmap = cm.jet)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(16,5))
ax1.scatter(lda_table1.x, lda_table1.y, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA1-PCA2')
ax2.scatter(lda_table1.y, lda_table1.z, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA2-PCA3')
ax3.scatter(lda_table1.z, lda_table1.x, c=class_train['Class'], marker = 'o', s=50, cmap = cm.jet, label='PCA3-PCA1')

In [ ]:
%%time
pca = PCA()
rfc = RandomForestClassifier(n_estimators=100, max_depth=50, n_jobs=7, random_state=33)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
pca.fit(X_train, y_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print(X_train.shape, X_train_pca.shape)
print(X_test.shape, X_test_pca.shape)

rfc.fit(X_train_pca, y_train)

accuracy = accuracy_score(y_test, rfc.predict(X_test_pca))
lloss = log_loss(y_test, rfc.predict_proba(X_test_pca), labels=list(range(1, 10)))

print('Accuracy %.3f' % accuracy)
print('Log Loss %.3f' % lloss)

In [ ]:
%%time
lda = LinearDiscriminantAnalysis()
rfc = RandomForestClassifier(n_estimators=100, max_depth=50, n_jobs=7, random_state=33)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
lda.fit(X_train, y_train)
X_train_lda = lda.transform(X_train)
X_test_lda = lda.transform(X_test)
print(X_train.shape, X_train_lda.shape)
print(X_test.shape, X_test_lda.shape)

rfc.fit(X_train_lda, y_train)

accuracy = accuracy_score(y_test, rfc.predict(X_test_lda))
lloss = log_loss(y_test, rfc.predict_proba(X_test_lda), labels=list(range(1, 10)))

print('Accuracy %.3f' % accuracy)
print('Log Loss %.3f' % lloss)